In [11]:
#first begin with imports
import warnings
warnings.filterwarnings("ignore")
import env
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pydataset import data
from statsmodels.formula.api import ols
import feature as fe

from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression, RFE
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [12]:
#load tips dataset and take a look
tips = data('tips')
tips

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3
241,27.18,2.00,Female,Yes,Sat,Dinner,2
242,22.67,2.00,Male,Yes,Sat,Dinner,2
243,17.82,1.75,Male,No,Sat,Dinner,2


1a.) Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [13]:
#making a new column which is tip divided by total bill and rounded to 2 decimal places
tips['tip_percentage'] = round(tips['tip']/tips['total_bill'],2)

In [14]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.06
2,10.34,1.66,Male,No,Sun,Dinner,3,0.16
3,21.01,3.50,Male,No,Sun,Dinner,3,0.17
4,23.68,3.31,Male,No,Sun,Dinner,2,0.14
5,24.59,3.61,Female,No,Sun,Dinner,4,0.15


In [15]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   total_bill      244 non-null    float64
 1   tip             244 non-null    float64
 2   sex             244 non-null    object 
 3   smoker          244 non-null    object 
 4   day             244 non-null    object 
 5   time            244 non-null    object 
 6   size            244 non-null    int64  
 7   tip_percentage  244 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 17.2+ KB


1b.) Create a column named price_per_person. This should be the total bill divided by the party size.

In [16]:
#make sure to use brackets since size is a keyword
tips['price_per_person'] = (tips['total_bill'])/(tips['size'])

In [17]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.06,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.16,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.17,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,0.14,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,0.15,6.147500



1c.) Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

I think the most important for tip amount is

size
time
I think the most important for tip percentage is

total bill
1d.) Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [18]:
target = 'tip'

In [19]:
X_train, y_train, X_validate, y_validate, X_test, y_test = fe.train_validate_test(tips, target)

In [20]:
obj_cols = fe.get_object_cols(X_train)
obj_cols

['sex', 'smoker', 'day', 'time']

In [21]:
num_cols = fe.get_numeric_X_cols(X_train, obj_cols)
num_cols

['total_bill', 'size', 'tip_percentage', 'price_per_person']

In [22]:
X_train_scaled, X_validate_scaled, X_test_scaled = fe.min_max_scale(X_train, X_validate, X_test, num_cols)

In [23]:
f_selector = SelectKBest(f_regression, k=2)

In [24]:
f_selector.fit(X_train_scaled, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7fdeb034f040>)

In [25]:
X_reduced = f_selector.transform(X_train_scaled)

print(X_train_scaled.shape)
print(X_reduced.shape)

(136, 4)
(136, 2)


In [26]:
#or condense the last 3 steps into this
X_reduced2 = SelectKBest(f_regression, k=2).fit_transform(X_train_scaled, y_train)
print(X_reduced2.shape)

(136, 2)


In [27]:
f_support = f_selector.get_support()

print(f_support)

[ True  True False False]


In [28]:
f_feature = X_train_scaled.loc[:,f_support].columns.tolist()

# you could also get the list this way (among many others)
# f_feature = [X_train_scaled.columns.values[i] for i in range(len(feature_mask)) if feature_mask[i]==True]

print(str(len(f_feature)), 'selected features')
print(f'The selected features are {f_feature} for tip amount')

2 selected features
The selected features are ['total_bill', 'size'] for tip amount


In [29]:
#recursive
lm=LinearRegression()
rfe = RFE(lm, 2)
X_rfe = rfe.fit_transform(X_train_scaled,y_train) 
lm.fit(X_rfe,y_train)

var_ranks = rfe.ranking_
var_names = X_train_scaled.columns.tolist()

pd.DataFrame({'Var': var_names, 'Rank': var_ranks})

,Var,Rank
0,total_bill,1
1,size,2
2,tip_percentage,1
3,price_per_person,3


In [30]:
#selected total_bill and tip_percentage for tip amount

1e.) Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [31]:
new_target = 'tip_percentage'

In [32]:
X_train, y_train, X_validate, y_validate, X_test, y_test = fe.train_validate_test(tips, new_target)
obj_cols = fe.get_object_cols(X_train)
num_cols = fe.get_numeric_X_cols(X_train, obj_cols)
X_train_scaled, X_validate_scaled, X_test_scaled = fe.min_max_scale(X_train, X_validate, X_test, num_cols)

X_reduced = SelectKBest(f_regression, k=2).fit_transform(X_train_scaled, y_train)
print(X_reduced.shape)

(136, 2)


In [33]:
f_support = f_selector.get_support()
f_feature = X_train_scaled.loc[:,f_support].columns.tolist()

print(str(len(f_feature)), 'selected features')
print(f'The selected features are {f_feature} for tip percentage')

2 selected features
The selected features are ['total_bill', 'tip'] for tip percentage


In [34]:
#recursive
lm=LinearRegression()
rfe = RFE(lm, 2)
X_rfe = rfe.fit_transform(X_train_scaled,y_train) 
lm.fit(X_rfe,y_train)

LinearRegression()

In [35]:
var_ranks = rfe.ranking_
var_names = X_train_scaled.columns.tolist()

pd.DataFrame({'Var': var_names, 'Rank': var_ranks})

,Var,Rank
0,total_bill,1
1,tip,1
2,size,2
3,price_per_person,3


In [36]:
#selected total_bill and tip as features for predicting tip percentage

1f.) Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

Select K Best uses correlation to score relationships and select the strongest using f-regression. RFE evaluates performance metrics, deletes the weakest, then keeps running it until it gets your desired number of returned features. Therefore, they can be different because of their different approaches. Both should come up with about the same results, but as you add more features, there may be discrepencies with how a regression is ranked vs performance metrics. If features are very close to each other, I believe the methods may optimize different features.



2.) Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [37]:
def select_kbest(X_train_scaled, target, n):
    X_reduced2 = SelectKBest(f_regression, k=n).fit_transform(X_train_scaled, y_train)
    f_support = f_selector.get_support()
    f_feature = X_train_scaled.loc[:,f_support].columns.tolist()
    print(str(len(f_feature)), 'selected features')
    print(f'The selected features are {f_feature}')

In [38]:
select_kbest(X_train_scaled, 'tip', 2)

2 selected features
The selected features are ['total_bill', 'tip']


In [39]:
select_kbest(X_train_scaled,'tip_percentage',2)

2 selected features
The selected features are ['total_bill', 'tip']


3.) Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [40]:
def rfe(X_train_scaled, target, n):
    lm=LinearRegression()
    rfe = RFE(lm, n)
    X_rfe = rfe.fit_transform(X_train_scaled,y_train) 
    lm.fit(X_rfe,y_train)
    mask = rfe.support_
    rfe_features = X_train_scaled.loc[:,mask].columns.tolist()
    
    print(str(len(rfe_features)), 'selected features')
    print(rfe_features)

In [41]:
rfe(X_train_scaled, 'tip',2)

2 selected features
['total_bill', 'tip']


In [42]:
rfe(X_train_scaled, 'tip_percentage',2)

2 selected features
['total_bill', 'tip']


4.) Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [43]:
swiss_df = data('swiss')

In [44]:

swiss_df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [45]:
swiss_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [46]:
target = 'Fertility'

In [47]:
X_train, y_train, X_validate, y_validate, X_test, y_test = fe.train_validate_test(swiss_df, target)

In [48]:
X_train

,Agriculture,Examination,Education,Catholic,Infant.Mortality
Rolle,60.8,16,10,7.72,16.3
Lavaux,73.0,19,9,2.84,20.0
Nyone,50.9,22,12,15.14,16.7
Conthey,85.9,3,2,99.71,15.1
Yverdon,49.5,15,8,6.10,22.5
Oron,71.2,12,1,2.40,21.0
Cossonay,69.3,22,5,2.82,18.7
St Maurice,75.9,9,9,99.06,17.8
Franches-Mnt,39.7,5,5,93.40,20.2
Orbe,54.1,20,6,4.20,15.3


In [49]:
y_train

Rolle           60.5
Lavaux          65.1
Nyone           56.6
Conthey         75.5
Yverdon         65.4
Oron            72.5
Cossonay        61.7
St Maurice      65.0
Franches-Mnt    92.5
Orbe            57.4
Sarine          82.9
La Chauxdfnd    65.7
Neuchatel       64.4
Monthey         79.4
Broye           83.8
Paysd'enhaut    72.0
Le Locle        72.7
Sion            79.3
Herens          77.3
Glane           92.4
Neuveville      76.9
Grandson        71.7
Vevey           58.3
Martigwy        70.5
Avenches        68.9
Name: Fertility, dtype: float64

In [50]:
obj_cols = fe.get_object_cols(X_train)
obj_cols

[]

In [51]:
num_cols = fe.get_numeric_X_cols(X_train, obj_cols)
num_cols

['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']

In [52]:
X_train_scaled, X_validate_scaled, X_test_scaled = fe.min_max_scale(X_train, X_validate, X_test, num_cols)

In [53]:
X_train_scaled

,Agriculture,Examination,Education,Catholic,Infant.Mortality
Rolle,0.647561,0.40625,0.290323,0.054508,0.122449
Lavaux,0.796341,0.50000,0.258065,0.004508,0.500000
Nyone,0.526829,0.59375,0.354839,0.130533,0.163265
Conthey,0.953659,0.00000,0.032258,0.997029,0.000000
Yverdon,0.509756,0.37500,0.225806,0.037910,0.755102
Oron,0.774390,0.28125,0.000000,0.000000,0.602041
Cossonay,0.751220,0.59375,0.129032,0.004303,0.367347
St Maurice,0.831707,0.18750,0.258065,0.990369,0.275510
Franches-Mnt,0.390244,0.06250,0.129032,0.932377,0.520408
Orbe,0.565854,0.53125,0.161290,0.018443,0.020408


In [54]:
fe.select_kbest(swiss_df.drop(columns=['Fertility']), swiss_df.Fertility, top=3)


['Examination', 'Education', 'Catholic']

In [55]:
fe.rfe(swiss_df.drop(columns=['Fertility']), swiss_df.Fertility,3)

3 selected features
['Examination', 'Education', 'Infant.Mortality']


In [56]:
features = fe.rfe(swiss_df.drop(columns=['Fertility']), swiss_df.Fertility,3)

3 selected features
['Examination', 'Education', 'Infant.Mortality']


In [57]:
y_validate

Moudon         65.0
Echallens      68.3
Moutier        85.8
Morges         65.5
Sierre         92.2
Boudry         70.4
Payerne        74.2
Lausanne       55.7
Rive Gauche    42.8
Courtelary     80.2
Rive Droite    44.7
Aubonne        66.9
Name: Fertility, dtype: float64